In [1]:
import os
import warnings
from datasets import load_dataset
import pandas as pd
from typing import List, Optional
from pydantic import BaseModel, ValidationError
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
mongo_uri = os.getenv('MONGO_URI')
OPENAI_API_KEY = os.getenv('OPENAI')

In [2]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split = "train")
dataset = dataset.take(100)
dataset_df = pd.DataFrame(dataset)
dataset_df.head(5)

print(dataset_df.columns)

Index(['_id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'property_type', 'room_type', 'bed_type',
       'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review',
       'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'guests_included', 'images', 'host', 'address',
       'availability', 'review_scores', 'reviews', 'weekly_price',
       'monthly_price', 'text_embeddings', 'image_embeddings'],
      dtype='object')


In [3]:
class Host(BaseModel):
  host_id : str
  host_url: str
  host_name: str
  host_location: str
  host_about: str
  host_response_time : Optional[str] = None
  host_thumbnail_url : str
  host_picture_url : str
  host_response_rate : Optional[int] = None
  host_is_superhost: bool
  host_has_profile_pic: bool
  host_identity_verified: bool

class Location(BaseModel):
  type : str
  coordinates : List[float]
  is_location_exact: bool

class Address(BaseModel):
  street: str
  government_area: str
  market: str
  country: str
  country_code: str
  location: Location

class Review(BaseModel):
  _id : str
  date: Optional[datetime] = None
  listing_id: str
  reviewer_id: str
  reviewer_name: Optional[str] = None
  comments: Optional[str] = None

class Listing(BaseModel):
  _id: int
  listing_url: str
  name: str
  summary: str
  space: str
  description: str
  neighborhood_overview: Optional[str] = None
  notes: Optional[str] = None
  transit: Optional[str] = None
  access: str
  interaction: Optional[str] = None
  house_rules: str
  property_type: str
  room_type: str
  bed_type: str
  minimum_nights: int
  maximum_nights: int
  cancellation_policy: str
  last_scraped: Optional[datetime] = None
  calendar_last_scraped: Optional[datetime] = None
  first_review: Optional[datetime] = None
  last_review: Optional[datetime] = None
  accommodates: int
  bedrooms: Optional[float] = 0
  beds: Optional[float] = 0
  number_of_reviews: int
  bathrooms: Optional[float] = 0
  amenities: List[str]
  price: int
  security_deposit: Optional[float] = None
  cleaning_fee: Optional[float] = None
  extra_people: int
  guests_included: int
  images: dict
  host: Host
  address: Address
  availability: dict
  review_scores: dict
  reviews: List[Review]
  text_embeddings: List[float]


In [4]:
records = dataset_df.to_dict(orient = 'records')
for record in records:
  for key, value in record.items():
    if isinstance(value, list):
      processed_list = [None if pd.isnull(v) else v for v in value]
      record[key] = processed_list
    else:
      if pd.isnull(value):
        record[key] = None

try:
  listings = [Listing(**record).dict() for record in records]
  print(listings[0].keys())
except ValidationError as e:
  print(e)


dict_keys(['listing_url', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'property_type', 'room_type', 'bed_type', 'minimum_nights', 'maximum_nights', 'cancellation_policy', 'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review', 'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms', 'amenities', 'price', 'security_deposit', 'cleaning_fee', 'extra_people', 'guests_included', 'images', 'host', 'address', 'availability', 'review_scores', 'reviews', 'text_embeddings'])


In [5]:
#use if ur cluster has data already and u r running this again
collection.delete_many({})

NameError: name 'collection' is not defined

In [6]:
from pymongo.mongo_client import MongoClient
from pymongo.operations import SearchIndexModel
database_name = "airbnb_dataset"
collection_name = "listings_reviews"

def get_mongo_client(mongo_uri):
  client = MongoClient(mongo_uri, appname ="devrel")
  print("Connection to MongoDB Successful")
  return client

mongo_client = get_mongo_client(mongo_uri)
db = mongo_client.get_database(database_name)
collection = db.get_collection(collection_name)

collection.delete_many({})

collection.insert_many(listings)
print("Data Ingestion into Mongo Complete")



/Users/pranamshetty/Developer/Projects/deeplearning-ai-notebooks/mongorag/venv/lib/python3.13/site-packages/pymongo/uri_parser.py:329: UserWarning: Unknown option: ssl_cert_reqs. Did you mean one of (sockettimeoutms, heartbeatfrequencyms, localthresholdms) or maybe a camelCase version of one? Refer to docstring.
  return get_validated_options(opts, warn)


Connection to MongoDB Successful
Data Ingestion into Mongo Complete


In [8]:
#Vector Search done here
text_embedding_field_name = "text_embeddings"
vector_search_index_name_text = "vector_index_text"
vector_search_index_model = SearchIndexModel ( 
    definition = {
        "mappings" : {
            "dynamic" : True,
            "fields" : {
                text_embedding_field_name : {
                    "dimensions" : 1536,
                    "similarity" : "cosine",
                    "type" : "knnVector",
                }
            },
        }
    },
    name = vector_search_index_name_text ,
)

In [12]:
index_exists = False
for index in collection.list_indexes():
    print(index)
    if index['name'] == vector_search_index_name_text:
        index_exists = True
        print(index_exists)
        break

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [15]:
import time
if not index_exists:
    try:
        result = collection.create_search_index(model = vector_search_index_model)
        print("Creating index ...")
        time.sleep(20)
        print("Index created successfully", result)
        print("Wait for a few minutes before conducting search")
    except Exception as e:
        print(f"Error creating a vector search index: {str(e)}")
else:
    print(f"Index '{vector_search_index_name_text}' already exists.")

        

Creating index ...
Index created successfully vector_index_text
Wait for a few minutes before conducting search


In [40]:
import openai
openai.api_key = OPENAI_API_KEY
def get_embedding(text):
    if not text or not isinstance(text, str):
        return None
    try:
        response = openai.embeddings.create(
        input = text, 
        model = "text-embedding-3-small", dimensions = 1536)
        embedding = response.data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None
        

In [45]:
def vector_search(user_query, db, collection, vector_index="vector_index_text"):
    query_embedding = get_embedding(user_query)
    if query_embedding is None:
        return "Invalid query or embedding generation failed."
    
    # Define the vector search stage
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index,
            "queryVector": query_embedding,
            "path": text_embedding_field_name,
            "numCandidates": 150,
            "limit": 20
        }
    }

    try:
        # Perform the vector search
        results = collection.aggregate([vector_search_stage])
        
        # Execute the explain command
        explain_query_execution = db.command(
            'explain', {
                'aggregate': collection.name,
                'pipeline': [vector_search_stage],
                'cursor': {}
            },
            verbosity='executionStats'
        )
        
        # Print the explain output for debugging
        print("Explain Query Execution Structure:", explain_query_execution)
        
        # Access the time statistics safely
        stages = explain_query_execution.get('stages', [])
        if stages and '$vectorSearch' in stages[0]:
            vector_search_explain = stages[0]['$vectorSearch']
            millis_elapsed = vector_search_explain.get('explain', {}).get('collectStats', {}).get('millisElapsed', "N/A")
            print(f"Total time taken: {millis_elapsed} milliseconds")
        else:
            print("Could not find 'millisElapsed' in the explain output.")
        
        return list(results)
    except Exception as e:
        print(f"Error during vector search: {str(e)}")
        return []


In [46]:
class SearchResultItem(BaseModel):
    name : str
    accommodates : Optional[int] = None
    address : Address
    summary : Optional[str] = None
    description : Optional[str] = None
    neighborhood_overview : Optional[str] = None
    notes : Optional[str] = None


In [55]:
from IPython.display import display, HTML
def handle_user_query(query, db, collection):
    get_knowledge = vector_search(query, db, collection)
    if not get_knowledge:
        return "No results found.", "No source information available."
    search_results_models = [
    SearchResultItem(**result)
    for result in get_knowledge
    ]
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])
    completion = openai.chat.completions.create(
        model = "gpt-4o-mini",
    messages = [
    {
    "role" : "system",
    "content" : "You are a airbnb listing recommendation system. "},
    {
    "role" : "user",
    "content" : f"Answer this user query: {query} with the following context:\n {search_results_df}"
    }
    ]
    )
    api_response = completion.choices[0].message.content
    print(f" - User Question:\n{query}\n")
    print(f" - System Response: \n{api_response}\n")
    display(HTML(search_results_df.to_html()))
    return api_response

In [56]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from resturants, can you recommend a place? 
Include a reason as to why you've chosen your selection.
"""
handle_user_query(query, db, collection)

Explain Query Execution Structure: {'explainVersion': '1', 'stages': [{'$vectorSearch': {'index': 'vector_index_text', 'queryVector': [-0.00903306994587183, -0.060913268476724625, -0.02551375888288021, 0.0017286738147959113, -0.022809168323874474, -0.00918628554791212, -0.054438237100839615, -0.005249306093901396, -0.001667054370045662, 0.006548309698700905, -0.052972692996263504, 0.028111767023801804, -0.04540516436100006, -0.002096724696457386, -0.010598535649478436, 0.02352861501276493, 0.0038437177427113056, 0.04876258969306946, 0.020344391465187073, 0.048975758254528046, 0.0063884323462843895, 0.02005128376185894, -0.10797049850225449, -0.029417430981993675, 0.02007792890071869, 0.021330302581191063, -0.030083587393164635, 0.024914219975471497, 0.020983900874853134, -0.03333442658185959, -0.003747125156223774, -0.025820191949605942, -0.020024636760354042, -0.020131222903728485, 0.023755108937621117, -0.009246239438652992, 0.020490946248173714, -0.017866292968392372, 0.022635966539

,name,accommodates,address,summary,description,neighborhood_overview,notes
0,Cozy house at Beyoğlu,2,"{'street': 'Beyoğlu, İstanbul, Turkey', 'government_area': 'Beyoglu', 'market': 'Istanbul', 'country': 'Turkey', 'country_code': 'TR', 'location': {'type': 'Point', 'coordinates': [28.95825, 41.03777], 'is_location_exact': False}}","Hello dear Guests, wellcome to istanbul. My House is 2+1 and at second floor. 1 privite room is for my international guests. House is Very close to Taksim Square. You can Walk in 30 minutes or you can take a bus. The bus stop is only 100 m from home. You can go Taksim, Eminönü, Karaköy, Kadıköy, Beyazıt, Sultanahmet easily from home. I have 1 bed, two people can sleep together. Second person should pay extra. You can use kitchen, bathroom, free Wifi, dishwasher, washing machine, Ironing.","Hello dear Guests, wellcome to istanbul. My House is 2+1 and at second floor. 1 privite room is for my international guests. House is Very close to Taksim Square. You can Walk in 30 minutes or you can take a bus. The bus stop is only 100 m from home. You can go Taksim, Eminönü, Karaköy, Kadıköy, Beyazıt, Sultanahmet easily from home. I have 1 bed, two people can sleep together. Second person should pay extra. You can use kitchen, bathroom, free Wifi, dishwasher, washing machine, Ironing. Safe, quite, big house, wiev, Central, near the bus stop. Kitchen, bathroom, room, sitting room, balcony, washing machine, dishwasher, Ironing. You can ask any questions with phone, (Hidden by Airbnb) or email. + (Phone number hidden by Airbnb) is my (Hidden by Airbnb) number. (Email hidden by Airbnb) is my email adress Beyoğlu / Centre of İstanbul It calls Hasköy area, near the Golden Horn Its very closed to bus stop and Balat, Taksim, Pierre Loti, Eminönü, Sultan Ahmet, Kadıköy Just enjoy your hol","Beyoğlu / Centre of İstanbul It calls Hasköy area, near the Golden Horn",Just enjoy your holiday
1,Downtown Oporto Inn (room cleaning),2,"{'street': 'Porto, Porto, Portugal', 'government_area': 'Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, Vitória', 'market': 'Porto', 'country': 'Portugal', 'country_code': 'PT', 'location': {'type': 'Point', 'coordinates': [-8.60867, 41.1543], 'is_location_exact': False}}","Tradicional building, with high ceilings next to City Hall or Trindade Subway station, at a short walking distance from the historic center of this beautiful city. R It is the property of a book novel writer.","Tradicional building, with high ceilings next to City Hall or Trindade Subway station, at a short walking distance from the historic center of this beautiful city. R It is the property of a book novel writer. Cozy, located near the most interesting points of the city to provide a nice stay, with a low budget. Has a gift shop to buy handicraft, books and other gifts, It is a Inn with groundfloor and 1st floor, with access to a small patio at groundfloor. You may reach us by e-mail - (Email hidden by Airbnb) or (Hidden by Airbnb) (Website hidden by Airbnb) Exciting, urban and dinamic, stay with us, near the center, and enjoy a unique stay! Next to Trindade subway station, and also to Faria Guimarães subway station. Near Praca da República, where there are several bus stop. No private parking.","Exciting, urban and dinamic, stay with us, near the center, and enjoy a unique stay!",No private parking.
2,Banyan Bungalow,2,"{'street': 'Waialua, HI, United States', 'government_area': 'North Shore Oahu', 'market': 'Oahu', 'country': 'United States', 'country_code': 'US', 'location': {'type': 'Point', 'coordinates': [-158.1602, 21.57561], 'is_location_exact': False}}",The place to be on the north shore is where you can be steps from the ocean and watch the stars at night. Our 2 acre property (with tropical greenhouses) hosts a quiet cottage with private driveway/private access.,"The place to be on the north shore is where you can be steps from the ocean and watch the stars at night. Our 2 acre property (with tropical greenhouses) ho

"I recommend the **Cozy house at Beyoğlu** in İstanbul, Turkey.\n\nReason: This charming listing is located in Beyoğlu, which is known for its warm and friendly atmosphere. The area is vibrant and culturally rich, making it an inviting choice for travelers. Additionally, it’s conveniently close to a variety of restaurants, cafes, and shops, allowing you to explore the local cuisine easily. The host's welcoming attitude, as indicated in the listing summary, adds to the cozy ambiance, making your stay not just comfortable, but also enjoyable."